# **Machine Learning Training**

### Initial Imports:

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", 300)

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from joblib import dump, load # Save Pre-Trained Model Using Joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set Initial Random State
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

#### Read in CSV as Pandas DataFrame:

In [3]:
# Set path to CSV and read in CSV
csv_path = Path('Returns and Signals.csv')
return_signal_df=pd.read_csv(csv_path)
# Set index as datetime object and drop extraneous columns
return_signal_df.set_index(pd.to_datetime(return_signal_df['Date'], infer_datetime_format=True), inplace=True)
return_signal_df.drop(columns=['Date'], inplace=True)
return_signal_df.head()

,Adj Close,Volume,Returns,ema_crossover_signal,vol_trend_signal,bollinger_signal,obv_crossover_signal,sentiment_signal
Date,,,,,,,,
2019-01-02,27.929413,11190300.0,0.027788,1.0,-1.0,0.0,1.0,0.0
2019-01-03,27.416327,10803900.0,-0.018371,1.0,1.0,0.0,-1.0,0.0
2019-01-04,28.045584,14593700.0,0.022952,1.0,1.0,0.0,1.0,0.0
2019-01-07,28.529629,13419400.0,0.017259,1.0,-1.0,0.0,1.0,0.0
2019-01-08,29.042719,8179700.0,0.017984,1.0,-1.0,0.0,1.0,0.0


#### Check Data Quality:

In [4]:
return_signal_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 504 entries, 2019-01-02 to 2020-12-30
Data columns (total 8 columns):
Adj Close               504 non-null float64
Volume                  504 non-null float64
Returns                 504 non-null float64
ema_crossover_signal    504 non-null float64
vol_trend_signal        504 non-null float64
bollinger_signal        504 non-null float64
obv_crossover_signal    504 non-null float64
sentiment_signal        504 non-null float64
dtypes: float64(8)
memory usage: 35.4 KB


### Creating X and Dependent Variables:

In [5]:
def variables(return_signal_df):
    '''
    this func takes in return_signal_df dataframe,
    then shiting X (singnals) variables down (to the future) by 1,
    (assuming intraday trading, the position auto closed at T+1),
    then constructing a new column for the dependent variable ("positive return"),
    return a modified return_signal_df.
    '''
    # Set X Variables/Features 
    global x_var_list
    x_var_list = ['ema_crossover_signal','vol_trend_signal','bollinger_signal','obv_crossover_signal','sentiment_signal'] # as a filter
    # Shift 1 down as signals comes after prices 
    return_signal_df[x_var_list] = return_signal_df[x_var_list].shift(1)
    # Construct the dependent variable where if daily return is greater than 0, then 1, else, 0.
    return_signal_df['Positive Return'] = np.where(return_signal_df['Returns'] > 0, 1.0, 0.0)
    return_signal_df.dropna(inplace=True)
    return return_signal_df

In [6]:
return_signal_df_modified = variables(return_signal_df)
return_signal_df_modified.head()

,Adj Close,Volume,Returns,ema_crossover_signal,vol_trend_signal,bollinger_signal,obv_crossover_signal,sentiment_signal,Positive Return
Date,,,,,,,,,
2019-01-03,27.416327,10803900.0,-0.018371,1.0,-1.0,0.0,1.0,0.0,0.0
2019-01-04,28.045584,14593700.0,0.022952,1.0,1.0,0.0,-1.0,0.0,1.0
2019-01-07,28.529629,13419400.0,0.017259,1.0,1.0,0.0,1.0,0.0,1.0
2019-01-08,29.042719,8179700.0,0.017984,1.0,-1.0,0.0,1.0,0.0,1.0
2019-01-09,29.323465,10607500.0,0.009667,1.0,-1.0,0.0,1.0,0.0,1.0


### Separate X and Y Training Datasets:
80% (training) 20% (testing) split

In [7]:
def separate(return_signal_df_modified):
    '''
    thif func takes in return_signal_df_modified,
    split the data into trian (70%) and test (30%),
    return trian and test data sets for the X and dependent variables.
    '''
    split = int(0.7 * len(return_signal_df_modified))
    # X Variable
    x_train = return_signal_df_modified[x_var_list][:split]
    x_test = return_signal_df_modified[x_var_list][split:]
    # Dependent Variable
    y_train = return_signal_df_modified['Positive Return'][:split]
    y_test = return_signal_df_modified['Positive Return'][split:]
    return x_train, x_test, y_train, y_test

In [8]:
x_train, x_test, y_train, y_test = separate(return_signal_df_modified)

In [9]:
x_train.head(2)

,ema_crossover_signal,vol_trend_signal,bollinger_signal,obv_crossover_signal,sentiment_signal
Date,,,,,
2019-01-03,1.0,-1.0,0.0,1.0,0.0
2019-01-04,1.0,1.0,0.0,-1.0,0.0


In [10]:
x_test.head(2)

,ema_crossover_signal,vol_trend_signal,bollinger_signal,obv_crossover_signal,sentiment_signal
Date,,,,,
2020-05-28,1.0,1.0,0.0,1.0,1.0
2020-05-29,1.0,1.0,0.0,1.0,0.0


In [11]:
y_train.head(2)

Date
2019-01-03    0.0
2019-01-04    1.0
Name: Positive Return, dtype: float64

In [12]:
y_test.head(3)

Date
2020-05-28    1.0
2020-05-29    1.0
2020-06-01    0.0
Name: Positive Return, dtype: float64

In [13]:
x_train_reshape = x_train.values.reshape((x_train.shape[0], x_train.shape[1],1))
x_test_reshape = x_test.values.reshape((x_test.shape[0], x_test.shape[1], 1))

### Machine Learning Models Training:

In [14]:
# Helper Function:
def analyse_model(model, predictions):
    '''
    this helper func takes in model, and predictions,
    to print accuracy score for traning and testing,
    to print the classification report for the test data,
    return a result dataframe which contains actual vs. predicted y data.
    '''
    # Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
    result = y_test.to_frame()
    result["Predicted Value"] = predictions
    
    # Print performance metrics:
    print("Accuracy score (training): {0:.4f}".format(accuracy_score(y_train, model.predict(x_train))))
    print("Accuracy score (testing): {0:.4f}".format(accuracy_score(y_test, predictions)))
    print(" ")
    print("Classification Report for the test data:")
    print(classification_report(y_test, predictions))
    return result

In [15]:
# Helper Function:
def analyse_RNN_model(RNN_model, predictions):
    '''
    this helper func takes in model, and predictions,
    to print accuracy score for traning and testing,
    to print the classification report for the test data,
    return a result dataframe which contains actual vs. predicted y data.
    '''
    # Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
    result = y_test.to_frame()
    result["Predicted Value"] = predictions
    
    # Print performance metrics:
    print("Accuracy score (training): {0:.4f}".format(accuracy_score(y_train, RNN_model.predict_classes(x_train_reshape))))
    print("Accuracy score (testing): {0:.4f}".format(accuracy_score(y_test, predictions)))
    print(" ")
    print("Classification Report for the test data:")
    print(classification_report(y_test, predictions))
    return result

#### 1. Random Forest Model:

**Hyperparameters:**

In [16]:
# 100 trees and 3 steps
n_estimators=100
max_depth=3

In [17]:
def RandomForest(x_train, y_train, x_test, y_test):
    '''
    thif func takes in x_train, y_train, x_test, y_test,
    runing Random Forest Classification model,
    return the RF_model result, and its predicted values (RF_predicitons)
    '''
    # Fit the model using just the training set:
    RF_model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=0)
    RF_model.fit(x_train, y_train)
    # Make a prediction of "y" values from the x test dataset
    RF_predictions = RF_model.predict(x_test)
    
    return RF_model, RF_predictions

In [18]:
RF_model, RF_predictions = RandomForest(x_train, y_train, x_test, y_test)
RF_result = analyse_model(RF_model, RF_predictions)
RF_result.head()

Accuracy score (training): 0.5682
Accuracy score (testing): 0.4636
 
Classification Report for the test data:
              precision    recall  f1-score   support

         0.0       0.44      0.55      0.49        71
         1.0       0.49      0.39      0.43        80

    accuracy                           0.46       151
   macro avg       0.47      0.47      0.46       151
weighted avg       0.47      0.46      0.46       151



,Positive Return,Predicted Value
Date,,
2020-05-28,1.0,0.0
2020-05-29,1.0,1.0
2020-06-01,0.0,1.0
2020-06-02,1.0,1.0
2020-06-03,1.0,0.0


#### 2. Gradient Boosting Model:

**Hyperparameters:**

In [19]:
# small learning rate means each tree learns a little from previous tree, 
# and learn more information from the new data.
learning_rate=0.5
n_estimators=20
max_depth=3

In [20]:
def GradientBoost(x_train, y_train, x_test, y_test):
    '''
    thif func takes in x_train, y_train, x_test, y_test,
    runing Gradient Boost Trees Classification model,
    return the GB_model result, and its predicted values (GB_predicitons)
    '''
    # Fit the model using just the training set:
    GB_model = GradientBoostingClassifier(n_estimators=n_estimators,
                                        learning_rate=learning_rate,
                                        max_features=5, # number of signals,
                                        max_depth=max_depth,
                                        random_state=0)
    GB_model.fit(x_train, y_train)
    # Make a prediction of "y" values from the x test dataset
    GB_predictions = GB_model.predict(x_test)   
    
    return GB_model, GB_predictions

In [21]:
GB_model, GB_predictions = GradientBoost(x_train, y_train, x_test, y_test)
GB_result = analyse_model(GB_model, GB_predictions)
GB_result.head()

Accuracy score (training): 0.5795
Accuracy score (testing): 0.4834
 
Classification Report for the test data:
              precision    recall  f1-score   support

         0.0       0.47      0.68      0.55        71
         1.0       0.52      0.31      0.39        80

    accuracy                           0.48       151
   macro avg       0.49      0.49      0.47       151
weighted avg       0.50      0.48      0.47       151



,Positive Return,Predicted Value
Date,,
2020-05-28,1.0,0.0
2020-05-29,1.0,1.0
2020-06-01,0.0,1.0
2020-06-02,1.0,1.0
2020-06-03,1.0,0.0


#### 3. LSTM RNN Model:

**Hyperparameters:**

In [22]:
# Number of neuron per layer
units = 8
# Dropout helps to prevent overfitting, it will randomly set some output to zero.
dropout_rate = 0.2 # will randomly drop 20% of the units
# smaller batch size is recommended
batch_size = 2
# Use at least 10 epochs
epochs = 20

In [23]:
# Mean Absolute Error (MAE)  mae
# categorical_crossentropy
# binary_crossentropy
# mean_squared_error

In [24]:
def LSTM(x_train_reshape, y_train, x_test_reshape, y_test):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    '''
    thif func takes in x_train_reshape, y_train, x_test_reshape, y_test,
    runing LSTM RNN model,
    return the LSTM_model and its predicted values (LSTM_predictions).
    '''
    # STEP I: Define the LSTM RNN model
    LSTM_model = Sequential()
    # Layer 1
    LSTM_model.add(LSTM(
        units=units,
        return_sequences=True,
    input_shape=(x_train_reshape.shape[1], x_train_reshape.shape[2]))
        )
    LSTM_model.add(Dropout(dropout_rate))
    # Layer 2
    LSTM_model.add(LSTM(units=units, return_sequences=True))
    LSTM_model.add(Dropout(dropout_rate))
    # Layer 3
    LSTM_model.add(LSTM(units=units))
    LSTM_model.add(Dropout(dropout_rate))
    # Output layer
    LSTM_model.add(Dense(1))
    
    # Step II: Compile the model
    LSTM_model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
    )
    # Summarize the model
    print(LSTM_model.summary())
    
    # Step III: Training the model
    LSTM_model.fit(
        x_train_reshape,
        y_train,
        epochs=epochs,
        shuffle=False, # to keep the sequential order of the data
        batch_size=batch_size,
        verbose=1,
    )
    # Step IV: Make a prediction of "y" values from the x test dataset
    LSTM_predictions = LSTM_model.predict_classes(x_test_reshape)
    
    return LSTM_model, LSTM_predictions

In [25]:
LSTM_model, LSTM_predicted = LSTM(x_train_reshape, y_train, x_test_reshape, y_test)
LSTM_result=analyse_RNN_model(LSTM_model,LSTM_predicted)
LSTM_result.head()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 8)              320       
_________________________________________________________________
dropout (Dropout)            (None, 5, 8)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 8)              544       
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 8)              0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 8)                 544       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 1)                 9

,Positive Return,Predicted Value
Date,,
2020-05-28,1.0,0
2020-05-29,1.0,0
2020-06-01,0.0,0
2020-06-02,1.0,0
2020-06-03,1.0,0


#### 4. GRU RNN Model:

In [26]:
def GRU(x_train_reshape, y_train, x_test_reshape, y_test):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
    '''
    thif func takes in x_train_reshape, y_train, x_test_reshape, y_test,
    runing GRU RNN model,
    return the GRU_model and its predicted values (GRU_predictions).
    '''
    # STEP I: Define the LSTM RNN model
    GRU_model = Sequential()
    # Layer 1
    GRU_model.add(GRU(
        units=units,
        return_sequences=True,
        input_shape=(x_train_reshape.shape[1], x_train_reshape.shape[2]))
        )
    GRU_model.add(Dropout(dropout_rate))
    # Layer 2
    GRU_model.add(LSTM(units=units, return_sequences=True))
    GRU_model.add(Dropout(dropout_rate))
    # Layer 3
    GRU_model.add(LSTM(units=units))
    GRU_model.add(Dropout(dropout_rate))
    # Output layer
    GRU_model.add(Dense(1))
    
    # Step II: Compile the model
    GRU_model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
    )
    # Summarize the model
    print(GRU_model.summary())
    
    # Step III: Training the model
    GRU_model.fit(
        x_train_reshape,
        y_train,
        epochs=epochs,
        shuffle=False, # to keep the sequential order of the data
        batch_size=batch_size,
        verbose=1,
    )
    # Step IV: Make a prediction of "y" values from the x test dataset
    GRU_predictions = GRU_model.predict_classes(x_test_reshape)
    
    return GRU_model, GRU_predictions

In [27]:
GRU_model, GRU_predictions = GRU(x_train_reshape, y_train, x_test_reshape, y_test)
GRU_result=analyse_RNN_model(GRU_model, GRU_predictions)
GRU_result.head()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 5, 8)              264       
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 8)              0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 8)              544       
_________________________________________________________________
dropout_4 (Dropout)          (None, 5, 8)              0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 8)                 544       
_________________________________________________________________
dropout_5 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

,Positive Return,Predicted Value
Date,,
2020-05-28,1.0,0
2020-05-29,1.0,0
2020-06-01,0.0,0
2020-06-02,1.0,0
2020-06-03,1.0,0
